In [1]:
# limpio la memoria
rm(list = ls()) # remove all objects
gc() # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,631893,33.8,1412184,75.5,985029,52.7
Vcells,1170410,9.0,8388608,64.0,1815603,13.9


In [2]:
require("data.table")
require("lightgbm")

Loading required package: data.table

Loading required package: lightgbm



In [3]:
setwd("~/buckets/b1")

In [4]:
# defino los parametros de la corrida, en una lista, la variable global  PARAM
#  muy pronto esto se leera desde un archivo formato .yaml
PARAM <- list()
PARAM$experimento <- "Ensemble Semillerío (Clase 14)"

PARAM$input$seeds <- "./exp/Ensemble Semillerío (Clase 14)/Ensemble Semillerío (Clase 14)_predicciones_semillas.csv"
PARAM$input$dataset <- "datasets/competencia_03_clase13.csv.gz"


# meses donde se entrena el modelo
PARAM$input$training <- c(202107, 202106, 202105, 202104, 202103, 202102, 202101,
 202012, 202011, 202010, 202009, 202008, 202002, 202001, 201912, 201911,
 201910, 201909, 201908, 201907, 201906, 201905, 201904, 201903) #Dos meses después del training de la parametrización
PARAM$input$future <- c(202109) # meses donde se aplica el modelo

In [7]:
fread("./exp/Ensemble Semillerío (Clase 14)/Ensemble Semillerío (Clase 14)_predicciones_semillas.csv")

numero_de_cliente,foto_mes,semilla_279511,semilla_280521,semilla_281531,semilla_282541,semilla_283551,semilla_284561,semilla_285571,semilla_286581,⋯,semilla_370420,semilla_371430,semilla_372440,semilla_373450,semilla_374460,semilla_375470,semilla_376480,semilla_377490,semilla_378500,semilla_379511
<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
29183981,202109,1.114514e-04,1.213842e-04,8.829319e-05,1.490291e-04,1.270646e-04,1.081678e-04,1.661754e-04,2.076182e-04,⋯,1.082913e-04,1.672554e-04,1.814711e-04,1.664548e-04,1.349504e-04,1.258579e-04,1.694567e-04,1.741908e-04,1.404136e-04,9.862840e-05
29184630,202109,3.847590e-04,7.236792e-04,3.815133e-04,5.822163e-04,4.826920e-04,6.151067e-04,4.196348e-04,3.714351e-04,⋯,5.362687e-04,4.501028e-04,2.867585e-04,7.928622e-04,4.203108e-04,4.822239e-04,4.622716e-04,5.416886e-04,6.276128e-04,3.467445e-04
29185433,202109,1.298310e-02,1.261656e-02,1.596465e-02,9.035416e-03,1.268999e-02,1.165785e-02,1.060167e-02,1.021276e-02,⋯,1.110769e-02,1.194421e-02,9.811555e-03,1.037684e-02,9.013167e-03,1.294627e-02,1.051531e-02,9.170241e-03,1.013671e-02,8.997077e-03
29185587,202109,2.926991e-04,2.597007e-04,1.843444e-04,2.033081e-04,2.311320e-04,1.620717e-04,1.975850e-04,1.987109e-04,⋯,2.561069e-04,2.284560e-04,2.170079e-04,3.065346e-04,1.778059e-04,2.721216e-04,1.924149e-04,2.478690e-04,2.233191e-04,2.576227e-04
29185646,202109,4.287096e-05,3.936486e-05,6.267073e-05,6.952677e-05,3.227417e-05,6.708642e-05,5.146100e-05,3.644713e-05,⋯,2.765228e-05,3.744395e-05,6.333765e-05,3.900243e-05,4.152625e-05,8.261415e-05,4.375392e-05,3.014447e-05,6.128974e-05,6.164538e-05
29187499,202109,7.044696e-04,8.203720e-04,6.544202e-04,8.975501e-04,7.303156e-04,6.590958e-04,5.179962e-04,8.016891e-04,⋯,4.955338e-04,4.362859e-04,5.806296e-04,7.172363e-04,7.866281e-04,4.731231e-04,6.325572e-04,1.008916e-03,4.460619e-04,8.663107e-04
29188277,202109,1.101277e-02,1.306722e-02,1.268777e-02,1.155128e-02,1.217016e-02,1.251896e-02,1.087308e-02,1.226169e-02,⋯,1.527870e-02,1.063203e-02,1.363447e-02,1.215443e-02,1.337811e-02,1.573501e-02,1.240991e-02,1.794258e-02,1.433122e-02,1.030874e-02
29189950,202109,1.288717e-03,1.715472e-03,2.025511e-03,1.724436e-03,1.984538e-03,1.559955e-03,1.064695e-03,1.917641e-03,⋯,1.618799e-03,2.381105e-03,1.103421e-03,1.946909e-03,1.464311e-03,1.683144e-03,1.429919e-03,1.459306e-03,2.302587e-03,1.490720e-03
29190599,202109,3.559525e-05,2.142541e-05,3.859703e-05,2.275033e-05,2.214335e-05,2.358291e-05,4.268309e-05,2.949062e-05,⋯,2.734136e-05,2.793534e-05,2.897240e-05,2.697783e-05,2.054922e-05,3.061786e-05,2.706317e-05,1.540923e-05,3.985032e-05,2.760551e-05


In [6]:
# Cargo archivo de predicciones
predicciones <- fread(cmd = PARAM$input$seeds)
datos <- fread(PARAM$input$dataset)
datos_reales <- datos[datos$foto_mes == 202109, c("numero_de_cliente", "foto_mes", "clase_ternaria")]

# Tranformo la clase ternaria en binaria
datos_reales[, real := ifelse(clase_ternaria %in% c("BAJA+2"), 1L, 0L)]

rm(datos)


In [ ]:
predicciones

In [ ]:
# Creo carpeta donde guardar los experimentos en caso de que no exista
dir.create("./exp/", showWarnings = FALSE)

# Creo carpeta donde guardar este experimento en caso de que no exista
dir.create(paste0("./exp/", PARAM$experimento, "/"), showWarnings = FALSE)

# Establezco el Working Directory de este experimento
setwd(paste0("./exp/", PARAM$experimento, "/"))

In [ ]:
# Armo Ensembles

# # Crear un df para almacenar los resultados del ensemble
ensemble_resultados <- data.frame(numero_de_cliente = predicciones$numero_de_cliente)

# Cambio nombre de columnas. Los nuevos nombres de columnas serán "semilla_1" hasta "semilla_100".
colnames(predicciones)[3:102] <- paste0("semilla_", 1:100)

# Me genero un ensemble (promedio de salidas) de las 100 semillas

# Crear un dataframe para almacenar los resultados del ensemble
ensemble_resultados <- data.frame(numero_de_cliente = predicciones$numero_de_cliente)

# Cambiar nombre de columnas (semilla_1 a semilla_100)
colnames(predicciones)[3:102] <- paste0("semilla_", 1:100)

# Calcular el promedio de las predicciones para los 100 modelos
ensemble_resultados$proba_ensemble <- rowMeans(predicciones[, 3:102])

In [ ]:
# Guardo el archivo
archivo_salida <- paste0(PARAM$experimento, "ensemble_resultados.csv")
fwrite(ensemble_resultados, file = archivo_salida, sep = ",")

In [ ]:
# Guardar el archivo como TXT
tb_entrega <- paste0(PARAM$experimento, "ensemble_resultados.txt")
write.table(ensemble_resultados, file = tb_entrega, sep = "\t", row.names = FALSE)

# ordeno por probabilidad descendente
setorder(tb_entrega, -prob)

In [ ]:
# genero archivos con los  "envios" mejores

cortes <- seq(8000, 15000, by = 500)
for (envios in cortes) {
  tb_entrega[, Predicted := 0L]
  tb_entrega[1:envios, Predicted := 1L]

  fwrite(tb_entrega[, list(numero_de_cliente, Predicted)],
    file = paste0(PARAM$experimento, "_", envios, ".csv"),
    sep = ","
  )
}

cat("\n\nLa generacion de los archivos para Kaggle ha terminado\n")